In [1]:
# EfficentNet : 2019 Google Brain
# 기존 모델보다 파라메터소를 줄이고 더 정확도를 높임
# 기존 CNN
    # 깊이 : ResNet ( 50 -> 101)
    # 너비 : Wide ResNet 각 층의 채널수를 늘린다
    # 해상도 : 224 -> 331
# 깊이 너비 해상도 -> 비율로 동시에 확장
# BaseLine MBConv

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import expit # 시그모이드 함수
# -10 ~ 10 까지 0.2 간격으로 생생
x = np.arange(-10, 10, 0.2)

# Relu 함수
# x.clip(0) # 0보다 작은 값은 0으로 변환, 0보다 크면 그대로


In [8]:
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from tqdm import tqdm
import torch
import torch.nn as nn


In [9]:
transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4,0.4,0.4),(0.2,0.2,0.2))
])

# 이미지 데이터 로드
train_dataset = torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transforms)
test_dataset = torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transforms)
trainloader = torch.utils.data.DataLoader(train_dataset,batch_size=64,shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset,batch_size=64,shuffle=False)
classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')
model = models.efficientnet_b0(weights =True )

c:\Users\31799\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\31799/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 63.4MB/s]


In [10]:
model.classifier[1] = nn.Linear(model.classifier[1].in_features,out_features=10)

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 손실함수와 최적화기 설정
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

# 모델 학습
for epoch in range(5):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(trainloader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/5], Loss: {running_loss/len(trainloader):.4f}')

  5%|▌         | 41/782 [02:13<40:16,  3.26s/it] 


KeyboardInterrupt: 